<a href="https://colab.research.google.com/github/chuckfinca/arithmephi/blob/main/arithmephi_evaluate_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==2.1.0
!pip install pyarrow==14.0.2
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
!pip install einops datasets bitsandbytes accelerate peft flash_attn num2words

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-mujcsqcm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-mujcsqcm
  Resolved https://github.com/huggingface/transformers to commit 0290ec19c901adc0f1230ebdccad11c40af026f5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8661022 sha256=60c12dbeb2ebc0d4362e6da04e49b5c5800c07704dfbd84045f5840d2cb2caec
  Stored in directory: /tmp/pip-ephem-wheel-cache-jt9a3n84/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.1 MB/s eta 0:00:00
     ━━━━

In [2]:
import flash_attn, torch, locale, pandas, time, random, string, num2words
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

In [3]:
# model = AutoModelForCausalLM.from_pretrained("chuckfinca/arithmephi", torch_dtype="auto", trust_remote_code=True, device_map = 'cuda')
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True, device_map = 'cuda')
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
def response(query):
  inputs = tokenizer(query, return_tensors="pt", return_attention_mask=False).to("cuda")
  outputs = model.generate(**inputs, max_length=len(inputs[0]) + 8, pad_token_id=tokenizer.eos_token_id)
  text = tokenizer.batch_decode(outputs)[0]
  return text

In [13]:
def counting_test(add_typos):
  max_int = 3000000
  prompt, answer = generate_counting_prompt_and_answer(-1 * max_int, max_int, add_typos)

  response_string = response(prompt).replace(prompt, "")
  return prompt, answer, response_string

In [6]:
def generate_counting_prompt_and_answer(min, max, add_typos):
    number = random.randint(min, max)
    prompt = ""
    for i in range(3):
        if prompt != "":
            prompt += " "

        # if random.getrandbits(1):
        #     prompt += num2words.num2words(float(number))
        #     if add_typos:
        #       prompt = insert_typos(prompt, 0.05)
        # else:
        #     prompt += f"{number}"
        prompt += f"{number}, "
        number += 1
    prompt = prompt[:-2]
    return prompt, f"{number}"

In [7]:
def insert_typos(phrase, probability_of_typo):
    # TODO: this can be made better using common misspellings like those found https://www.dcs.bbk.ac.uk/~ROGER/corpora.html
    new_phrase = []
    words = phrase.split(' ')
    for word in words:
        outcome = random.random()
        if outcome <= probability_of_typo:
            typo_type = random.choice(['insert', 'omit', 'replace', 'replace'])
            ix = random.choice(range(len(word)))
            if typo_type == 'replace':
                new_word = ''.join([word[w] if w != ix else random.choice(string.ascii_letters) for w in range(len(word))])
            elif typo_type == 'insert':
                new_word = ''.join([word[w] if w != ix else word[w] + random.choice(string.ascii_letters) for w in range(len(word))]) + (random.choice(string.ascii_letters) if ix == len(word) - 1 else '')
            elif typo_type == 'omit' and len(word) > 1:
                new_word = ''.join([word[w] for w in range(len(word)) if w != ix])
            else:
                new_word = word
            new_phrase.append(new_word)
        else:
            new_phrase.append(word)

    return ' '.join([w for w in new_phrase])

In [8]:
def extract_number(response):
  response_array = response.replace("<|endoftext|>", "").split(", ")
  answer = response_array[1] if len(response_array) > 1 else response
  array = answer.split(" ")
  if array[0] == '':
    array = array[1:]
  try:
    return int(array[0].split("\n")[0].split(",")[0])
  except:
    return response.split("\n")[0].split(",")[0]

In [15]:
start_time = time.time()

# Initialize lists
prompts = []
answers = []
extracted_responses = []

score = 0
number_of_questions = 100
for i in range(number_of_questions):
  prompt, answer, extracted_response = counting_test(False)
  print(f"iteration: {i}")

  print(f"prompt: {prompt}")
  print(f"answer: {answer}")

  number_response = extract_number(extracted_response)
  print(f"extracted_response: {number_response}")
  try:
    if int(number_response) == int(answer):
      score += 1
    else:
      print("❌")
  except:
      print("❌")


  # prompt, answer, extracted_response = counting_test()
  prompts.append(prompt)
  answers.append(answer)
  extracted_responses.append(extract_number(extracted_response))

print(f"Score: {score}/{number_of_questions}")

# Create DataFrame
df = pandas.DataFrame({
    'Prompt': prompts,
    'Answer': answers,
    'Extracted Response': extracted_responses
})

iteration: 0
prompt: -1557979,  -1557978,  -1557977
answer: -1557976
extracted_response: -1557975
❌
iteration: 1
prompt: 1352259,  1352260,  1352261
answer: 1352262
extracted_response: 1352262
iteration: 2
prompt: 1700879,  1700880,  1700881
answer: 1700882
extracted_response: 1700882
iteration: 3
prompt: 1187965,  1187966,  1187967
answer: 1187968
extracted_response: 1187968
iteration: 4
prompt: -2319729,  -2319728,  -2319727
answer: -2319726
extracted_response: -2319726
iteration: 5
prompt: 2969380,  2969381,  2969382
answer: 2969383
extracted_response: 2969383
iteration: 6
prompt: 1698939,  1698940,  1698941
answer: 1698942
extracted_response: 1698942
iteration: 7
prompt: -2250081,  -2250080,  -2250079
answer: -2250078
extracted_response: -2250078
iteration: 8
prompt: 352385,  352386,  352387
answer: 352388
extracted_response: 352388
iteration: 9
prompt: 2138669,  2138670,  2138671
answer: 2138672
extracted_response: 2138670
❌
iteration: 10
prompt: -360886,  -360885,  -360884
answer